In [0]:
from __future__ import print_function
import torch
import torch.nn as nn # Neural networks can be constructed using the torch.nn package.
import torch.nn.functional as F 
import torch.optim as optim #package implementing various optimization algorithms
from torchvision import datasets, transforms #The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.

In [0]:
class Net(nn.Module): #An nn.Module contains layers, and a method forward(input)that returns the output.
    def __init__(self):
        super(Net, self).__init__()# supposed to initialize the layers you want to use
        self.conv1 = nn.Conv2d(1, 16, 3,padding = 1) #input Channel - 1, OUtput Channel - 32,  RF - 3
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding =1 ) #input Channel - 32, OUtput Channel - 64,  RF - 5
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout2d(0.1)
        self.conv3 = nn.Conv2d(32, 16, 1) #input Channel - 64, OUtput Channel - 32,  RF - 10
        self.bn3 = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1) #input Channel - 32, OUtput Channel - 64,  RF - 12
        self.bn4 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.drop2 = nn.Dropout2d(0.1)
        self.conv5 = nn.Conv2d(32, 32, 3,padding = 1) #input Channel - 32, OUtput Channel - 64,  RF - 26
        self.bn5 = nn.BatchNorm2d(32)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.conv7 = nn.Conv2d(32, 10, 1) 
       
        

    def forward(self, x): #In the forward method, you specify the connections of your layers
        #x = self.drop1(self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))) #Relu non linear activaion function
        #x = self.drop2(self.pool2(F.relu(self.conv4(F.relu(self.conv3(x))))))
        x = self.drop1(self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))) #Relu non linear activaion function
        x = self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x)))))))
        x = F.relu(self.bn5(self.conv5(x)))
        #x = F.relu(self.conv9(F.relu(self.conv8(x))))
        x = F.relu(self.conv7(x))
        x = self.avgpool(x)
        #print(x.shape)
        x = x.view(-1, 10)
        return F.log_softmax(x,dim=1)

In [14]:
!pip install torchsummary #Yes, you can get exact Keras representation, using pytorch-summary package.
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 32, 28, 28]           4,640
       BatchNorm2d-4           [-1, 32, 28, 28]              64
         MaxPool2d-5           [-1, 32, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]             528
       BatchNorm2d-7           [-1, 16, 14, 14]              32
            Conv2d-8           [-1, 32, 14, 14]           4,640
       BatchNorm2d-9           [-1, 32, 14, 14]              64
        MaxPool2d-10             [-1, 32, 7, 7]               0
           Conv2d-11             [-1, 32, 7, 7]           9,248
      BatchNorm2d-12             [-1, 32, 7, 7]              64
           Conv2d-13             [-1, 10, 7, 7]             330
AdaptiveAvgPool2d-14             [-1, 1

In [0]:


torch.manual_seed(1) #Sets the seed for generating random numbers (seed (int) – The desired seed.)
batch_size = 8
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader( 
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=False, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data) 
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [20]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(18):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/7500 [00:00<?, ?it/s]


Test set: Average loss: 0.0553, Accuracy: 9826/10000 (98.260%)



  0%|          | 0/7500 [00:00<?, ?it/s]


Test set: Average loss: 0.0418, Accuracy: 9860/10000 (98.600%)



  0%|          | 0/7500 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0211, Accuracy: 9935/10000 (99.350%)



  0%|          | 0/7500 [00:00<?, ?it/s]


Test set: Average loss: 0.0178, Accuracy: 9940/10000 (99.400%)



loss=0.001291513442993164 batch_id=7499: 100%|██████████| 7500/7500 [01:02<00:00, 119.71it/s]



Test set: Average loss: 0.0205, Accuracy: 9932/10000 (99.320%)

